In [2]:
import yfinance as yf
import pandas as pd
import os
import warnings
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sb
from newsapi import NewsApiClient
from newspaper import Article
import nltk
from concurrent.futures import ThreadPoolExecutor
import requests

BASE_DIR="/Users/dannyyu/Desktop/AI_Trader/data"
news_api_key="7ff3cc2cda3c46c18558bcec8a1e498d"
newsapi = NewsApiClient(api_key=news_api_key)
TICKER_KEYWORDS = {
    "AAPL":  '"Apple Inc" OR "AAPL" OR "Apple"',
    "MSFT":  '"Microsoft Corporation" OR "MSFT" OR "Microsoft"',
    "GOOGL": '"Google" OR "Alphabet Inc" OR "GOOGL"',
    "AMZN":  '"Amazon.com Inc" OR "Amazon" OR "AMZN"',
    "META":  '"Meta Platforms" OR "Facebook" OR "META" OR "Meta Platforms Inc"',
    "NVDA":  '"NVIDIA Corporation" OR "Nvidia" OR "NVDA"',
    "TSLA":  '"Tesla Inc" OR "Tesla" OR "TSLA"',
    "INTC":  '"Intel Corporation" OR "Intel" OR "INTC"',
    "AMD":   '"Advanced Micro Devices" OR "AMD" OR "AMD Inc"',
    "IBM":   '"International Business Machines" OR "IBM"'
}
BASE_URL = "https://newsapi.org/v2/everything"
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'INTC', 'AMD', 'IBM']

In [21]:
def extract_text(url, max_retries=3):
    for attempt in range(max_retries):
        try:
            art = Article(url, fetch_images=False, memoize_articles=False)
            art.download()
            art.parse()
            return art.text
        except Exception as e:
            print(f"[{attempt+1}/{max_retries}] Failed to fetch {url}: {e}")
            continue  # Try again
    return None

def get_full_content(df):
    urls=df["url"]
    with ThreadPoolExecutor(max_workers=15) as pool:
        texts = list(pool.map(extract_text, urls))

    return texts

def read_CSV_File(ticker,folderName):
    dir = BASE_DIR+"/"+folderName+"/"+ticker+"_"+folderName+".csv"
    table= pd.read_csv(dir,index_col=0,low_memory=False)
    return table

def missing_values(table,ticker):
    total_rows = len(table)
    hasMissing=False
    print("Analyzing "+ticker)
    print(table.size)
    for column in table.columns:
        num_missing = table[column].isna().sum()
        if num_missing == 0:
            #print(f"📊 '{column}': no missing values")
            continue  # skip columns with no missing data
        else:
            hasMissing=True
        percent_missing = 100 * num_missing / total_rows
        print(f"📊 '{column}': {num_missing} missing ({percent_missing:.2f}%)")
    if not hasMissing:
        print(ticker + " has no missing values")
    return hasMissing

In [217]:
os.makedirs(os.path.join(BASE_DIR, "newsapi"), exist_ok=True)

In [23]:
for ticker in TICKERS:
    df=read_CSV_File(ticker,"newsapi")
    missing_values(df,ticker)
    #df = df[df["full_content"].notna() & (df["full_content"].str.strip() != "")]
    #df.to_csv(os.path.join(BASE_DIR, "newsapi",f"{ticker}_cleaned_newsapi.csv"), index=True)

Analyzing AAPL
525
📊 'author': 1 missing (1.33%)
📊 'description': 1 missing (1.33%)
📊 'full_content': 1 missing (1.33%)
Analyzing MSFT
525
📊 'author': 6 missing (8.00%)
📊 'description': 6 missing (8.00%)
📊 'full_content': 11 missing (14.67%)
Analyzing GOOGL
525
GOOGL has no missing values
Analyzing AMZN
525
📊 'author': 1 missing (1.33%)
Analyzing META
525
📊 'author': 15 missing (20.00%)
📊 'description': 15 missing (20.00%)
📊 'full_content': 18 missing (24.00%)
Analyzing NVDA
525
📊 'author': 41 missing (54.67%)
📊 'description': 41 missing (54.67%)
📊 'full_content': 45 missing (60.00%)
Analyzing TSLA
525
📊 'author': 22 missing (29.33%)
📊 'description': 22 missing (29.33%)
📊 'full_content': 23 missing (30.67%)
Analyzing INTC
525
📊 'author': 12 missing (16.00%)
📊 'description': 11 missing (14.67%)
📊 'full_content': 14 missing (18.67%)
Analyzing AMD
525
📊 'author': 18 missing (24.00%)
📊 'description': 19 missing (25.33%)
📊 'full_content': 23 missing (30.67%)
Analyzing IBM
525
📊 'author': 23

In [19]:
for ticker, keyword in TICKER_KEYWORDS.items():
    print(f"Processing {ticker}")
    params = {
            "q": keyword,
            "searchIn": "title",
            "from": "2025-05-13",
            "to": "2025-06-12",
            "language": "en",
            "sortBy": "relevancy",
            "pageSize": 75,
            "page": page,
            "apiKey": news_api_key
        }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    all_articles=data.get("articles", [])
    df = pd.DataFrame(all_articles)
    
    df=df.drop(columns=['urlToImage'], errors="ignore")
    df['full_content'] = get_full_content(df)
    df["publishedAt"] = df["publishedAt"].apply(
        lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").date()
    )
    df = df.sort_values(by="publishedAt").reset_index(drop=True)
    df.to_csv(os.path.join(BASE_DIR, "newsapi",f"{ticker}_newsapi.csv"), index=False)




Processing AAPL


KeyError: 'url'

In [199]:
params = {
            "q": keyword,
            "searchIn": "title",
            "from": "2025-05-13",
            "to": "2025-06-12",
            "language": "en",
            "sortBy": "relevancy",
            "pageSize": 75,
            "page": 1,
            "apiKey": news_api_key
        }
response1 = requests.get(BASE_URL, params=params)


In [213]:

response1
df=pd.DataFrame(response1["articles"])
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'the-verge', 'name': 'The Verge'}",Emma Roth,Apple WWDC 2025: the 13 biggest announcements,Apple packed a bunch of major updates into its...,https://www.theverge.com/news/682769/apple-wwd...,https://platform.theverge.com/wp-content/uploa...,2025-06-09T19:04:31Z,Apple packed a bunch of major updates into its...
1,"{'id': 'wired', 'name': 'Wired'}",David Nield,How to Use Apple Maps on the Web,Apple’s mapping platform is no longer exclusiv...,https://www.wired.com/story/how-to-use-apple-m...,https://media.wired.com/photos/681d1858c4e2ebb...,2025-05-13T12:00:00Z,The boundaries ofApples walled garden arent as...
2,"{'id': 'the-verge', 'name': 'The Verge'}",Emma Roth,Trump wants Apple to stop making more iPhones ...,President Donald Trump told Apple CEO Tim Cook...,https://www.theverge.com/news/667649/donald-tr...,https://platform.theverge.com/wp-content/uploa...,2025-05-15T14:18:30Z,Were not interested in you building in India.\...
3,"{'id': 'the-verge', 'name': 'The Verge'}",Jacob Kastrenakes,Apple ordered to keep web links in the App Store,Apple will have to continue allowing web links...,https://www.theverge.com/news/679946/apple-rej...,https://platform.theverge.com/wp-content/uploa...,2025-06-04T21:53:46Z,An Appeals Court rejected Apples request to bl...
4,"{'id': 'the-verge', 'name': 'The Verge'}",Allison Johnson,All the Android features Apple announced at WWDC,Apple announcing Android features years after ...,https://www.theverge.com/apple-wwdc/686247/app...,https://platform.theverge.com/wp-content/uploa...,2025-06-12T18:49:04Z,A little Dub Dub deja vu.\r\nApple announcing ...
...,...,...,...,...,...,...,...,...
70,"{'id': None, 'name': 'CNET'}",Bree Fowler,Apple Says Workout Buddy Will Cheer You On,Great Job! A new Apple Watch feature will prov...,https://www.cnet.com/health/fitness/apple-says...,https://www.cnet.com/a/img/resize/f34da8651ae1...,2025-06-09T18:28:00Z,Apple says its new Workout Buddy feature will ...
71,"{'id': None, 'name': 'MacRumors'}",Hartley Charlton,Report: Apple Preparing to Launch Mind-Control...,Apple is planning to allow users to natively c...,https://www.macrumors.com/2025/05/13/apple-pre...,https://images.macrumors.com/t/hkHwk6kjRla1Urq...,2025-05-13T13:18:40Z,Apple is planning to allow users to natively c...
72,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Apple Outlines WWDC 2025 Labs and Community Ev...,In addition to sharing a complete WWDC 2025 sc...,https://www.macrumors.com/2025/05/20/wwdc-2025...,https://images.macrumors.com/t/2zBnbBELX2QLJEf...,2025-05-20T16:45:00Z,In addition to sharing a complete WWDC 2025 sc...
73,"{'id': None, 'name': 'MacRumors'}",Tim Hardwick,Apple Smart Glasses: Everything We Know So Far,Google made waves yesterday by showcasing a se...,https://www.macrumors.com/2025/05/21/apple-sma...,https://images.macrumors.com/t/EvkvdK3fPdSpGS8...,2025-05-21T15:21:02Z,Google made waves yesterday by showcasing a se...


In [57]:
response1 = newsapi.get_everything(
        q=keyword,
        from_param="2025-05-13",
        searchIn="title",
        to="2025-06-12",
        language="en",
        sort_by="relevancy",
        page_size=75,
        page=1)
    response2 = newsapi.get_everything(
        q=keyword,
        from_param="2025-05-13",
        searchIn="title",
        to="2025-06-12",
        language="en",
        sort_by="relevancy",
        page_size=75,
        page=2)

""
